# Gaussian mixtures
In this notebook we fit gaussian mixtures to the 25 stocks, using 2 components, then 3 components.

## Two-component models

In [13]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture


stocks = np.load('data/plr_stocks.npy')

model1 = GaussianMixture(n_components=2, tol=1e-5, n_init=10, max_iter=500)
model1.fit(stocks[0].reshape(-1, 1))

weights = model1.weights_        # shape (K,)
means = model1.means_.ravel()    # shape (K,)
variances = model1.covariances_.ravel()  # shape (K,) in 1D if covariance_type='full'

print("weights:", weights)
print("means:", means)
print("variances:", variances)
print('aic : ', model1.aic(stocks[0].reshape(-1, 1)), "\n")

model_test_init = GaussianMixture(n_components=2, tol=1e-5, weights_init=[0.49, 0.51], means_init=[[0], [0]])
model_test_init.fit(stocks[0].reshape(-1, 1))
print(model_test_init.weights_)
print(model_test_init.means_)
print(model_test_init.covariances_.ravel())


weights: [0.06368054 0.93631946]
means: [-0.68026214  0.08382701]
variances: [82.91478302  4.34575356]
aic :  37524.16384078409 

[0.97819832 0.02180168]
[[3.59530396e-02]
 [9.32552444e-06]]
[9.59299856e+00 1.01611973e-06]


In [14]:
paramsfit2c = []
resultsfit2c = []
i = 1
number_inits = 150
import os
os.environ["OMP_NUM_THREADS"] = "1"


print('Fitting 2 component gaussian mixtures to the 25 stocks...')
print(f'Number of initializations : {number_inits} \n')

for stock in stocks:
    gmm2c = GaussianMixture(n_components = 2, tol = 1e-4, n_init = number_inits, init_params='random_from_data', random_state = 42)
    gmm2c.fit(stock.reshape(-1, 1))

    weights = gmm2c.weights_
    means = gmm2c.means_
    variances = gmm2c.covariances_.ravel()

    paramsfit2c.append({'stock': i, 'pi1': weights[0], 'pi2' : weights[1],
                        'mu1': means[0, 0], 'mu2' : means[1, 0],
                        'sigma1':np.sqrt(variances[0]), 'sigma2': np.sqrt(variances[1])})
    
    loglikelihood = gmm2c.score(stock.reshape(-1, 1)) * len(stock.reshape(-1, 1))
    aic = gmm2c.aic(stock.reshape(-1, 1))
    bic = gmm2c.bic(stock.reshape(-1, 1))

    resultsfit2c.append({'stock':i, 'log-likelihood':loglikelihood,
                         'aic':aic, 'bic':bic})

    i += 1

# saving results
results = pd.DataFrame(resultsfit2c)
fitted_params = pd.DataFrame(paramsfit2c)

print(results.head())
fitted_params.head()

results.to_csv('results/gmm2c_results.csv', index = False)
fitted_params.to_csv('results/gmm2c_fitted_params.csv', index = False)


Fitting 2 component gaussian mixtures to the 25 stocks...
Number of initializations : 150 

   stock  log-likelihood           aic           bic
0      1   -18756.867798  37523.735597  37558.671581
1      2   -16706.517249  33423.034498  33457.970482
2      3   -16265.720830  32541.441660  32576.377644
3      4   -16469.902528  32949.805055  32984.741039
4      5   -14562.856240  29135.712480  29170.648464


## Value at Risk estimation for 2-component model

In [15]:
from scipy.stats import Normal, Mixture

print(results.head())
#print(results.iloc[0]['mu'])

results_var2c = []

for i in range(results.shape[0]):

    pi1 = fitted_params.iloc[i]['pi1']
    pi2 = fitted_params.iloc[i]['pi2']
    mu1 = fitted_params.iloc[i]['mu1']
    mu2 = fitted_params.iloc[i]['mu2']
    sigma1 = fitted_params.iloc[i]['sigma1']
    sigma2 = fitted_params.iloc[i]['sigma2']

    alpha = 0.01
    dist1 = Normal(mu = mu1, sigma = sigma1)
    dist2 = Normal(mu = mu2, sigma = sigma2)
    mixt = Mixture(components = [dist1, dist2], weights = [pi1, pi2])

    empirical_var = -np.percentile(stocks[i], alpha * 100)
    var = -mixt.icdf(alpha)
    results_var2c.append({'stock': i + 1,'estimated-var':var, 'empirical-var':empirical_var})

    
results_var2c = pd.DataFrame(results_var2c)
results_var2c.head()
results_var2c.to_csv('results/ValueAtRiskResults/gmm2c_var.csv', index=False)


   stock  log-likelihood           aic           bic
0      1   -18756.867798  37523.735597  37558.671581
1      2   -16706.517249  33423.034498  33457.970482
2      3   -16265.720830  32541.441660  32576.377644
3      4   -16469.902528  32949.805055  32984.741039
4      5   -14562.856240  29135.712480  29170.648464


## Three component models

In [17]:
paramsfit3c = []
resultsfit3c = []
i = 1

print('Fitting 3 component gaussian mixtures to the 25 stocks...')
print(f'Number of initializations : {number_inits} \n')

for stock in stocks:
    gmm3c = GaussianMixture(n_components = 3, tol = 1e-4, n_init = number_inits, init_params='random_from_data', random_state=42)
    gmm3c.fit(stock.reshape(-1, 1))

    weights = gmm3c.weights_
    means = gmm3c.means_
    variances = gmm3c.covariances_.ravel()

    paramsfit3c.append({'stock': i, 'pi1': weights[0], 'pi2' : weights[1], 'pi3':weights[2],
                        'mu1': means[0, 0], 'mu2' : means[1, 0], 'mu3': means[2, 0],
                        'sigma1':np.sqrt(variances[0]), 'sigma2': np.sqrt(variances[1]), 
                        'sigma3': np.sqrt(variances[2])})
    
    loglikelihood = gmm3c.score(stock.reshape(-1, 1)) * len(stock.reshape(-1, 1))
    aic = gmm3c.aic(stock.reshape(-1, 1))
    bic = gmm3c.bic(stock.reshape(-1, 1))

    resultsfit3c.append({'stock':i, 'log-likelihood':loglikelihood,
                         'aic':aic, 'bic':bic})

    i += 1

# saving results
results = pd.DataFrame(resultsfit3c)
fitted_params = pd.DataFrame(paramsfit3c)

print(results.head())
print(fitted_params.head())

results.to_csv('results/gmm3c_results.csv', index = False)
fitted_params.to_csv('results/gmm3c_fitted_params.csv', index = False)


Fitting 3 component gaussian mixtures to the 25 stocks...
Number of initializations : 150 

   stock  log-likelihood           aic           bic
0      1   -18241.499103  36498.998206  36554.895781
1      2   -16096.057018  32208.114037  32264.011611
2      3   -16176.394268  32368.788536  32424.686110
3      4   -16430.558003  32877.116005  32933.013580
4      5   -14161.773383  28339.546766  28395.444340
   stock       pi1       pi2       pi3       mu1           mu2           mu3  \
0      1  0.920595  0.057727  0.021678  0.084438 -7.373305e-01  9.149753e-06   
1      2  0.741492  0.026871  0.231637  0.070491 -1.456026e-18 -1.427837e-01   
2      3  0.018624  0.709069  0.272307 -0.831341  6.980971e-02 -6.786708e-02   
3      4  0.047214  0.469338  0.483448 -0.649331  5.440386e-02  7.253401e-02   
4      5  0.878287  0.101205  0.020509  0.055344 -2.482671e-01 -3.193605e-16   

     sigma1    sigma2    sigma3  
0  2.136531  9.440568  0.001008  
1  1.297750  0.001000  3.996397  
2  8.45

# Value at Risk estimation for three-component model

In [18]:
from scipy.stats import Normal, Mixture

print(results.head())

results_var3c = []

for i in range(fitted_params.shape[0]):

    pi1 = fitted_params.iloc[i]['pi1']
    pi2 = fitted_params.iloc[i]['pi2']
    pi3 = fitted_params.iloc[i]['pi3']
    mu1 = fitted_params.iloc[i]['mu1']
    mu2 = fitted_params.iloc[i]['mu2']
    mu3 = fitted_params.iloc[i]['mu3']
    sigma1 = fitted_params.iloc[i]['sigma1']
    sigma2 = fitted_params.iloc[i]['sigma2']
    sigma3 = fitted_params.iloc[i]['sigma3']

    alpha = 0.01
    dist1 = Normal(mu = mu1, sigma = sigma1)
    dist2 = Normal(mu = mu2, sigma = sigma2)
    dist3 = Normal(mu = mu3, sigma = sigma3)
    mixt = Mixture(components = [dist1, dist2, dist3], weights = [pi1, pi2, pi3])

    empirical_var = -np.percentile(stocks[i], alpha * 100)
    var = -mixt.icdf(alpha)
    print('Empirical var:', empirical_var)
    print('estimated var:',var, '\n')
    results_var3c.append({'stock': i + 1,'estimated-var':var, 'empirical-var':empirical_var})

    
results_var3c = pd.DataFrame(results_var3c)
results_var3c.head()
results_var3c.to_csv('results/ValueAtRiskResults/gmm3c_var.csv', index=False)


   stock  log-likelihood           aic           bic
0      1   -18241.499103  36498.998206  36554.895781
1      2   -16096.057018  32208.114037  32264.011611
2      3   -16176.394268  32368.788536  32424.686110
3      4   -16430.558003  32877.116005  32933.013580
4      5   -14161.773383  28339.546766  28395.444340
Empirical var: 7.460909595509617
estimated var: 9.627047731801621 

Empirical var: 6.033780936051005
estimated var: 6.996677473854746 

Empirical var: 5.821528465112555
estimated var: 5.846612252764375 

Empirical var: 5.680408769439981
estimated var: 5.634903911625886 

Empirical var: 4.167985044887263
estimated var: 5.071143563471284 

Empirical var: 5.013481919836497
estimated var: 5.637357366837136 

Empirical var: 5.45530787598142
estimated var: 6.136059202816525 

Empirical var: 5.002146995149504
estimated var: 5.3575932864406175 

Empirical var: 6.846430997702966
estimated var: 7.36868235652209 

Empirical var: 3.6742151568336774
estimated var: 3.8958476347030313 

E